<a href="https://colab.research.google.com/github/ML-Bioinfo-CEITEC/cDNA-pretraining/blob/main/data/DNA_data_K6_stride1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installation

In [1]:
!pip install -qq transformers datasets

     |████████████████████████████████| 4.7 MB 4.7 MB/s 
     |████████████████████████████████| 365 kB 32.6 MB/s 
     |████████████████████████████████| 6.6 MB 41.9 MB/s 
     |████████████████████████████████| 120 kB 47.1 MB/s 
     |████████████████████████████████| 115 kB 37.3 MB/s 
     |████████████████████████████████| 212 kB 66.7 MB/s 
     |████████████████████████████████| 127 kB 61.5 MB/s 


In [2]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [21]:
### PARAMETERS

# K in K-MERS, currently implemented for K=6 only
K = 6
# data source
input_dataset = "davidcechak/Zebrafish_DNA_v0"
# the name for HF Hub output
output_name = f"Zebrafish_DNA_v0_tokenized_kmer{K}_stride1"

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(f"armheb/DNA_bert_6")

In [23]:
from datasets import load_dataset

splitted_datasets = load_dataset(input_dataset)

Downloading:   0%|          | 0.00/691 [00:00<?, ?B/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/davidcechak___parquet/davidcechak--Zebrafish_DNA_v0-16cff65d8552e4e0/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]


## Stride 1

In [24]:
def kmers_stride1(s, k=K):
    return [s[i:i + k] for i in range(0, len(s)-k+1)]

kmers_stride1("ATGGAAAGAGGCACCATTCT")    

['ATGGAA',
 'TGGAAA',
 'GGAAAG',
 'GAAAGA',
 'AAAGAG',
 'AAGAGG',
 'AGAGGC',
 'GAGGCA',
 'AGGCAC',
 'GGCACC',
 'GCACCA',
 'CACCAT',
 'ACCATT',
 'CCATTC',
 'CATTCT']

In [25]:
def tokenize_function(s, k=K):
  seq_split = " ".join(kmers_stride1(s['Seq'], k))
  return tokenizer(seq_split)

tokenize_function({'Seq':'ACCTGCTGGACGATCATA'})

{'input_ids': [2, 675, 2686, 2540, 1956, 3713, 2551, 2000, 3889, 3254, 715, 2845, 3174, 393, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [26]:
tokenized_datasets = splitted_datasets.map(tokenize_function, remove_columns='Seq', num_proc=4)
tokenized_datasets

#0:   0%|          | 0/3091 [00:00<?, ?ex/s]

#1:   0%|          | 0/3090 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (9997 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (9997 > 512). Running this sequence through the model will result in indexing errors


#2:   0%|          | 0/3090 [00:00<?, ?ex/s]

#3:   0%|          | 0/3090 [00:00<?, ?ex/s]

...

#2:   0%|          | 0/27810 [00:01<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (9997 > 512). Running this sequence through the model will result in indexing errors


#3:   0%|          | 0/27810 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (9997 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (9997 > 512). Running this sequence through the model will result in indexing errors


DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 12361
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 111241
    })
})

In [27]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py

def group_texts(examples, max_length=512):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result

chunked_datasets = tokenized_datasets.map(group_texts, batched=True, desc=f"Grouping texts in chunks of 512")
chunked_datasets

Grouping texts in chunks of 512:   0%|          | 0/13 [00:00<?, ?ba/s]

Grouping texts in chunks of 512:   0%|          | 0/112 [00:00<?, ?ba/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 241348
    })
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2171980
    })
})

In [28]:
chunked_datasets.push_to_hub(output_name)

Pushing dataset shards to the dataset hub:   0%|          | 0/2 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/14 [00:00<?, ?it/s]

In [29]:
# Arabidopsis_thaliana - 209152
# Worm - 176154
# FruitFly - 238634
# ZebraFish - 2171980